In [1]:
import pandas as pd

pd.__version__

'2.3.3'

In [2]:
df = pd.read_parquet(
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2024-01.parquet"
)

In [3]:
import sklearn
sklearn.__version__

'1.7.2'

In [4]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2024-01-01 00:57:55,2024-01-01 01:17:43,1.0,1.72,1.0,N,186,79,2,17.70,1.00,0.5,0.00,0.00,1.0,22.70,2.5,0.0
1,1,2024-01-01 00:03:00,2024-01-01 00:09:36,1.0,1.80,1.0,N,140,236,1,10.00,3.50,0.5,3.75,0.00,1.0,18.75,2.5,0.0
2,1,2024-01-01 00:17:06,2024-01-01 00:35:01,1.0,4.70,1.0,N,236,79,1,23.30,3.50,0.5,3.00,0.00,1.0,31.30,2.5,0.0
3,1,2024-01-01 00:36:38,2024-01-01 00:44:56,1.0,1.40,1.0,N,79,211,1,10.00,3.50,0.5,2.00,0.00,1.0,17.00,2.5,0.0
4,1,2024-01-01 00:46:51,2024-01-01 00:52:57,1.0,0.80,1.0,N,211,148,1,7.90,3.50,0.5,3.20,0.00,1.0,16.10,2.5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,2024-01-31 23:45:59,2024-01-31 23:54:36,NaN,3.18,NaN,None,107,263,0,15.77,0.00,0.5,2.00,0.00,1.0,21.77,NaN,NaN
2964620,1,2024-01-31 23:13:07,2024-01-31 23:27:52,NaN,4.00,NaN,None,114,236,0,18.40,1.00,0.5,2.34,0.00,1.0,25.74,NaN,NaN
2964621,2,2024-01-31 23:19:00,2024-01-31 23:38:00,NaN,3.33,NaN,None,211,25,0,19.97,0.00,0.5,0.00,0.00,1.0,23.97,NaN,NaN
2964622,2,2024-01-31 23:07:23,2024-01-31 23:25:14,NaN,3.06,NaN,None,107,13,0,23.88,0.00,0.5,5.58,0.00,1.0,33.46,NaN,NaN


In [5]:
df.isna().sum()


VendorID                      0
tpep_pickup_datetime          0
tpep_dropoff_datetime         0
passenger_count          140162
trip_distance                 0
RatecodeID               140162
store_and_fwd_flag       140162
PULocationID                  0
DOLocationID                  0
payment_type                  0
fare_amount                   0
extra                         0
mta_tax                       0
tip_amount                    0
tolls_amount                  0
improvement_surcharge         0
total_amount                  0
congestion_surcharge     140162
Airport_fee              140162
dtype: int64

In [12]:
col_to_del = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'RatecodeID', 'store_and_fwd_flag', 'payment_type', 'congestion_surcharge', 'Airport_fee', 'PULocationID', 'DOLocationID']

In [7]:
df['duration_sec'] = (
    df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
).dt.total_seconds()

dt = df['tpep_pickup_datetime']
df['hour'] = dt.dt.hour
df['dayofweek'] = dt.dt.dayofweek
df['is_weekend'] = (df['dayofweek'] >= 5).astype(int)

df['PU_DO'] = (
    df['PULocationID'].astype(str)
    + '_'
    + df['DOLocationID'].astype(str)
)

In [7]:
results = []

for col in df.columns:
    if col == 'passenger_count':
        continue
    if not pd.api.types.is_numeric_dtype(df[col]):
        continue
    
    null_both = df[['passenger_count', col]].isna().all(axis=1).sum()
    
    df_clean = df[
        df['passenger_count'].notna() &
        df[col].notna()
    ]
    
    corr = df_clean['passenger_count'].corr(df[col])
    
    results.append({
        'column': col,
        'null_both': null_both,
        'correlation': corr
    })

result_df = pd.DataFrame(results).sort_values(
    by='correlation', key=abs, ascending=False
)

print(result_df)


                   column  null_both  correlation
0                VendorID          0     0.098827
7                   extra          0    -0.050986
2              RatecodeID     140162    -0.038366
6             fare_amount          0     0.036255
12           total_amount          0     0.035230
10           tolls_amount          0     0.029482
15           duration_sec          0     0.021718
14            Airport_fee     140162     0.018451
5            payment_type          0     0.016380
9              tip_amount          0     0.015381
1           trip_distance          0     0.011788
8                 mta_tax          0    -0.010787
3            PULocationID          0    -0.009012
13   congestion_surcharge     140162     0.008886
4            DOLocationID          0    -0.005117
11  improvement_surcharge          0     0.000597


In [15]:
df.groupby('passenger_count')['total_amount'].agg(
    mean_total_amount='mean',
    trip_count='count'
)

,mean_total_amount,trip_count
passenger_count,,
0.0,25.327817,31465
1.0,26.205230,2188739
2.0,29.520660,405103
3.0,29.138309,91262
4.0,30.877267,51974
5.0,26.269129,33506
6.0,25.801183,22353
7.0,57.735000,8
8.0,95.668039,51


In [8]:
# After checking correlation -> not need to predict passenger_count from other data, just using mode (the value(s) that appear most often.)
mode_passenger_count_df = df.copy()
mode_passenger_count_df['passenger_count'] = df['passenger_count'].fillna(df['passenger_count'].mode()[0])

In [9]:
mode_passenger_count_df['passenger_count'].isna().sum()

np.int64(0)

In [13]:
# NaN passenger_count -> Use other correlation like total_amount, airport_fee

# Feature to use: passenger_count, RatecodeID, PULocationID, DOLocationID, total amount

cleaned_df = mode_passenger_count_df.drop(columns=col_to_del)

In [14]:
cleaned_df

,VendorID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,duration_sec,hour,dayofweek,is_weekend,PU_DO
0,2,1.0,1.72,17.70,1.00,0.5,0.00,0.00,1.0,22.70,1188.0,0,0,0,186_79
1,1,1.0,1.80,10.00,3.50,0.5,3.75,0.00,1.0,18.75,396.0,0,0,0,140_236
2,1,1.0,4.70,23.30,3.50,0.5,3.00,0.00,1.0,31.30,1075.0,0,0,0,236_79
3,1,1.0,1.40,10.00,3.50,0.5,2.00,0.00,1.0,17.00,498.0,0,0,0,79_211
4,1,1.0,0.80,7.90,3.50,0.5,3.20,0.00,1.0,16.10,366.0,0,0,0,211_148
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2964619,2,1.0,3.18,15.77,0.00,0.5,2.00,0.00,1.0,21.77,517.0,23,2,0,107_263
2964620,1,1.0,4.00,18.40,1.00,0.5,2.34,0.00,1.0,25.74,885.0,23,2,0,114_236
2964621,2,1.0,3.33,19.97,0.00,0.5,0.00,0.00,1.0,23.97,1140.0,23,2,0,211_25
2964622,2,1.0,3.06,23.88,0.00,0.5,5.58,0.00,1.0,33.46,1071.0,23,2,0,107_13


In [15]:
from sklearn.model_selection import train_test_split


target_col = 'duration_sec'

X = cleaned_df.drop(columns=[target_col])
y = cleaned_df[target_col]

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

In [16]:
from catboost import CatBoostRegressor

cat_cols = [
    'VendorID',
    'hour',
    'dayofweek',
    'is_weekend',
    'PU_DO',
]

cat_feature_indices = [X.columns.get_loc(col) for col in cat_cols]

model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    random_seed=42,
    verbose=100
)

model.fit(
    X_train, y_train,
    cat_features=cat_feature_indices,
    eval_set=(X_test, y_test),
    use_best_model=True
)

0:	learn: 2114.0008126	test: 1948.3598076	best: 1948.3598076 (0)	total: 1.13s	remaining: 9m 24s
100:	learn: 2009.4423483	test: 1842.1643654	best: 1842.1643654 (100)	total: 1m 10s	remaining: 4m 39s
200:	learn: 2004.1880449	test: 1839.7652419	best: 1839.7652419 (200)	total: 2m 21s	remaining: 3m 31s
300:	learn: 2000.1282357	test: 1838.3335342	best: 1838.3335342 (300)	total: 3m 30s	remaining: 2m 19s
400:	learn: 1992.7267257	test: 1837.6262394	best: 1837.5995939 (395)	total: 4m 36s	remaining: 1m 8s
499:	learn: 1988.1131929	test: 1837.2338351	best: 1837.2338351 (499)	total: 5m 39s	remaining: 0us

bestTest = 1837.233835
bestIteration = 499



In [15]:
cleaned_df['duration_sec'].quantile([0.5, 0.9, 0.99, 0.999])


0.500     698.000
0.900    1732.000
0.990    3627.000
0.999    6892.262
Name: duration_sec, dtype: float64

In [ ]:
y_pred = model.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

rmse = mean_squared_error(y_test, y_pred, squared=False)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)
print("R²:", r2)